In [160]:
import plotly.express as px
import pandas as pd
import numpy as np
import json

In [145]:
infl2 = pd.read_csv('inflation.csv')
def us_convert(before, now, value):
	bef_cpi = infl2[infl2.year == before].cpi.values[0]
	now_cpi = infl2[infl2.year == now].cpi.values[0]
	perc = (now_cpi - bef_cpi) / bef_cpi * 100
	print(perc)
	return value * (1 + perc / 100)
	

In [251]:
movies = pd.read_csv("movies_metadata.csv", dtype={'popularity': 'str'})
movies = movies.drop(columns=["adult", "poster_path", "homepage", "video", "original_title", "belongs_to_collection"])
movies = movies.dropna(subset=["release_date"])
movies["release_year"] = movies["release_date"].str[:4]

section1 = movies.drop(columns=["imdb_id", "runtime", "revenue", "vote_count", "vote_average", "status", "popularity", "tagline", 'overview'])

In [302]:
df = section1.drop(columns=['production_companies', 'production_countries', 'title', 'spoken_languages', 'release_date'])
df.head()

,budget,genres,id,original_language,release_year
0,30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862,en,1995
1,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",8844,en,1995
2,0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",15602,en,1995
3,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",31357,en,1995
4,0,"[{'id': 35, 'name': 'Comedy'}]",11862,en,1995


In [303]:
def convert_genres(s):
	l = json.loads(s.replace('\'', '"'))
	ret = []
	for d in l:
		ret.append(d['name'])
	return ret

In [304]:
df.genres = df.genres.apply(convert_genres)

In [305]:
# df = df.explode('genres')
df = df.drop(columns=['id', 'original_language'])
df

,budget,genres,release_year
0,30000000,"[Animation, Comedy, Family]",1995
1,65000000,"[Adventure, Fantasy, Family]",1995
2,0,"[Romance, Comedy]",1995
3,16000000,"[Comedy, Drama, Romance]",1995
4,0,[Comedy],1995
...,...,...,...
45457,0,"[Drama, Action, Romance]",1991
45459,0,[Drama],2011
45460,0,"[Action, Drama, Thriller]",2003
45461,0,[],1917


In [334]:
df = df[~(df.budget == 0)]
df = df[~(df.release_year.astype(int) < 1950)]
df

,budget,genres,release_year
0,30000000,"[Animation, Comedy, Family]",1995
1,65000000,"[Adventure, Fantasy, Family]",1995
3,16000000,"[Comedy, Drama, Romance]",1995
5,60000000,"[Action, Crime, Drama, Thriller]",1995
6,58000000,"[Comedy, Romance]",1995
...,...,...,...
45399,2000000,[Comedy],2014
45405,2000000,[Comedy],2004
45406,800000,"[Comedy, Drama]",2006
45409,2000000,"[Romance, Drama]",2010


In [335]:
exploded = df.explode('genres')
exploded

,budget,genres,release_year
0,30000000,Animation,1995
0,30000000,Comedy,1995
0,30000000,Family,1995
1,65000000,Adventure,1995
1,65000000,Fantasy,1995
...,...,...,...
45409,2000000,Drama,2010
45419,5000000,Action,2007
45419,5000000,Comedy,2007
45419,5000000,Crime,2007


In [336]:
inputs_by_genres = exploded.groupby(['release_year', 'genres']).mean().reset_index()
inputs_by_genres = inputs_by_genres.sort_values(by='release_year').reset_index(drop=True)
inputs_by_genres

,release_year,genres,budget
0,1950,Action,3768785.0
1,1950,Western,3768785.0
2,1950,Mystery,250000.0
3,1950,Music,2941892.5
4,1950,Fantasy,2900000.0
...,...,...,...
1170,2018,Drama,12000000.0
1171,2018,Fantasy,18000000.0
1172,2018,Action,18000000.0
1173,2018,Comedy,15000000.0


In [337]:
fig = px.line(inputs_by_genres, x='release_year', y='budget', color='genres')
fig.show()